# Basic Framework
First attempt to create a graph and a clustering of the words

In [1]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd
from tqdm import tqdm
# Math
import numpy as np
import scipy.stats
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa
import librosa.display
from scipy import sparse, stats, spatial
import scipy.sparse.linalg

from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd



%matplotlib inline

## 1. Import the Data
----
Use `N` random samples for each word 

In [30]:
N = 100

train_audio_path = '../Project/data/train/audio'
dirs = [f for f in os.listdir(train_audio_path) if isdir(join(train_audio_path, f))]
dirs.sort()

path = []
word = []
speaker = []
iteration = []

for direct in dirs:
    if not direct.startswith('_'):
        # Random selection of N files per folder 
        list_files = os.listdir(join(train_audio_path, direct))
        wave_selected  = list(np.random.choice([ f for f in list_files if f.endswith('.wav')],N,replace=False))
        
        # Extraction of file informations for dataframe
        word.extend(list(np.repeat(direct,N,axis=0)))
        speaker.extend([wave_selected[f].split('.')[0].split('_')[0] for f in range(N) ])
        iteration.extend([wave_selected[f].split('.')[0].split('_')[-1] for f in range(N) ])
        path.extend([train_audio_path + '/' + direct + '/' + wave_selected[f] for f in range(N)])


Initialize the dataframe of the original data

In [31]:
features_og = pd.DataFrame({('info','word',''): word,
                            ('info','speaker',''): speaker,
                            ('info','iteration',''): iteration,
                            ('info','path',''): path})
index_og = [('info','word',''),('info','speaker',''),('info','iteration','')]
#features_og.set_index(index_og,inplace=True)
features_og.head()

info                                                                  
  iteration                                               path   speaker word
                                                                             
0         0  ../Project/data/train/audio/bed/7fd25f7c_nohas...  7fd25f7c  bed
1         0  ../Project/data/train/audio/bed/a1a59b72_nohas...  a1a59b72  bed
2         0  ../Project/data/train/audio/bed/982babaf_nohas...  982babaf  bed
3         0  ../Project/data/train/audio/bed/105a0eea_nohas...  105a0eea  bed
4         0  ../Project/data/train/audio/bed/c6ca5d01_nohas...  c6ca5d01  bed

## 2. Features Extraction
----
### 2.1 MFCC
A classical but reliable set a features

In [32]:
N_MFCC = 20

def compute_mfcc(filepath):
    audio, sampling_rate = librosa.load(filepath, sr=None, mono=True)
    return librosa.feature.mfcc(y=audio,sr=sampling_rate)

In [33]:
stat_name= ['mean','std','skew','kurtosis','median']
col_names = [('mfcc',stat_name[i],j) for i in range(len(stat_name))  for j in range(N_MFCC)]
features_mfcc =pd.DataFrame(columns=pd.MultiIndex.from_tuples(col_names),index=features_og.index)
# sorting the columns in order to improve index performances (see lexsort errors)
features_mfcc.sort_index(axis=1,inplace=True,sort_remaining=True)

# MFCC FEATURES :
for w in tqdm(range(len(features_og)),total=len(features_og),unit='waves'):
    mfcc = compute_mfcc(features_og[('info','path')].iloc[w])
    features_mfcc.loc[w, ('mfcc', 'mean')] = np.mean(mfcc,axis=1)
    features_mfcc.loc[w, ('mfcc', 'std')] = np.std(mfcc,axis=1)
    features_mfcc.loc[w, ('mfcc', 'skew')] = scipy.stats.skew(mfcc,axis=1)
    features_mfcc.loc[w, ('mfcc', 'kurtosis')] = scipy.stats.kurtosis(mfcc,axis=1)
    features_mfcc.loc[w, ('mfcc', 'median')] = np.median(mfcc,axis=1)

features_og = features_og.merge(features_mfcc,left_index=True,right_index=True)
features_og.head()

100%|██████████| 3000/3000 [03:06<00:00, 16.12waves/s]


info                                                               \
     iteration                                               path   speaker   
                                                                              
0            0  ../Project/data/train/audio/bed/7fd25f7c_nohas...  7fd25f7c   
1            0  ../Project/data/train/audio/bed/a1a59b72_nohas...  a1a59b72   
2            0  ../Project/data/train/audio/bed/982babaf_nohas...  982babaf   
3            0  ../Project/data/train/audio/bed/105a0eea_nohas...  105a0eea   
4            0  ../Project/data/train/audio/bed/c6ca5d01_nohas...  c6ca5d01   
5            0  ../Project/data/train/audio/bed/d1a4fb3f_nohas...  d1a4fb3f   
6            1  ../Project/data/train/audio/bed/d84829e0_nohas...  d84829e0   
7            0  ../Project/data/train/audio/bed/3e3fa7fd_nohas...  3e3fa7fd   
8            0  ../Project/data/train/audio/bed/18c54a68_nohas...  18c54a68   
9            0  ../Project/data/train/audio/bed/e2362167_nohas...  e2362167   
10           1  ../Project/data/train/audio/bed/28612180_nohas...  28612180   
11           1  ../Project/data/train/audio/bed/4abb2400_nohas...  4abb2400   
12           0  ../Project/data/train/audio/bed/8830e17f_nohas...  8830e17f   
13           1  ../Project/data/train/audio/bed/fbf3dd31_nohas...  fbf3dd31   
14           0  ../Project/data/train/audio/bed/dabd70c2_nohas...  dabd70c2   
15           0  ../Project/data/train/audio/bed/80c45ed6_nohas...  80c45ed6   
16           1  ../Project/data/train/audio/bed/988e2f9a_nohas...  988e2f9a   
17           0  ../Project/data/train/audio/bed/1890e5c3_nohas...  1890e5c3   
18           0  ../Project/data/train/audio/bed/1bb6ed89_nohas...  1bb6ed89   
19           1  ../Project/data/train/audio/bed/f104f742_nohas...  f104f742   
20           0  ../Project/data/train/audio/bed/3e31dffe_nohas...  3e31dffe   
21           0  ../Project/data/train/audio/bed/6f3458b3_nohas...  6f3458b3   
22           0  ../Project/data/train/audio/bed/5a3712c9_nohas...  5a3712c9   
23           1  ../Project/data/train/audio/bed/d78858d9_nohas...  d78858d9   
24           0  ../Project/data/train/audio/bed/c25a032b_nohas...  c25a032b   
25           0  ../Project/data/train/audio/bed/2313e093_nohas...  2313e093   
26           0  ../Project/data/train/audio/bed/84bf12ff_nohas...  84bf12ff   
27           0  ../Project/data/train/audio/bed/a3255f5c_nohas...  a3255f5c   
28           0  ../Project/data/train/audio/bed/30f31e42_nohas...  30f31e42   
29           0  ../Project/data/train/audio/bed/747e69fd_nohas...  747e69fd   
...        ...                                                ...       ...   
2970         0  ../Project/data/train/audio/zero/de6eb27c_noha...  de6eb27c   
2971         1  ../Project/data/train/audio/zero/02746d24_noha...  02746d24   
2972         1  ../Project/data/train/audio/zero/4634529e_noha...  4634529e   
2973         0  ../Project/data/train/audio/zero/12c206ea_noha...  12c206ea   
2974         0  ../Project/data/train/audio/zero/a24582a0_noha...  a24582a0   
2975         4  ../Project/data/train/audio/zero/46a153d8_noha...  46a153d8   
2976         1  ../Project/data/train/audio/zero/61a0d340_noha...  61a0d340   
2977         0  ../Project/data/train/audio/zero/8d8d9855_noha...  8d8d9855   
2978         0  ../Project/data/train/audio/zero/a518d1cf_noha...  a518d1cf   
2979         2  ../Project/data/train/audio/zero/d933bf5e_noha...  d933bf5e   
2980         0  ../Project/data/train/audio/zero/b575b5fb_noha...  b575b5fb   
2981         1  ../Project/data/train/audio/zero/3e3b65eb_noha...  3e3b65eb   
2982         0  ../Project/data/train/audio/zero/54d9ccb5_noha...  54d9ccb5   
2983         0  ../Project/data/train/audio/zero/aac5b7c1_noha...  aac5b7c1   
2984         1  ../Project/data/train/audio/zero/cf68fad2_noha...  cf68fad2   
2985         4  ../Project/data/train/audio/zero/e7ea8b76_noha...  e7ea8b76   
2986         0  ../Project/data/train/audio/zero/d7529c0c_no

Saving the dataset features into a pickle to avoid to redo the computation on the training set

In [35]:
features_og.to_pickle('./Features Data/trainingFeatures.pickle')

## 3. Analysis

Importing the pickle containing the previously saved features

In [36]:
features_og = pd.read_pickle('./Features Data/trainingFeatures.pickle')

In [ ]:
# drop features
features = pd.DataFrame(features_og)
features -= features.mean(axis=0)
features /= features.std(axis=0)

distances = spatial.distance.squareform(spatial.distance.pdist(features,'cosine'))

n=distances.shape[0]
kernel_width = distances.mean()
weights = np.exp(np.divide(-np.square(distances),kernel_width**2))
np.fill_diagonal(weights,0)

In [ ]:
fix, axes = plt.subplots(2, 2, figsize=(17, 8))
def plot(weights, axes):
    axes[0].spy(weights)
    axes[1].hist(weights[weights > 0].reshape(-1), bins=50);
plot(weights, axes[:, 0])

NEIGHBORS = 30

for i in range(weights.shape[0]):
    idx = weights[i,:].argsort()[:-NEIGHBORS]
    weights[i,idx] = 0
    weights[idx,i] = 0

plot(weights, axes[:, 1])
   

In [ ]:
degrees = np.sum(weights,axis=0)
laplacian = np.diag(degrees**-0.5) @ (np.diag(degrees) - weights) @ np.diag(degrees**-0.5)
laplacian = sparse.csr_matrix(laplacian)
plt.matshow(laplacian.todense());

In [ ]:
eigenvalues, eigenvectors = sparse.linalg.eigsh(A=laplacian,k=10,which='SM')

plt.plot(eigenvalues, '.-', markersize=15);

x = eigenvectors[:,1]
y = eigenvectors[:,2]

In [ ]:
fix, axes = plt.subplots(5, 5, figsize=(17, 8))
for i in range(1,6):
    for j in range(1,6):
        x = eigenvectors[:,i]
        y = eigenvectors[:,j]
        labels = np.sign(x)
        axes[i-1,j-1].scatter(x, y, c=labels, cmap='RdBu', alpha=0.5)